In [ ]:
from IPython.display import display, Markdown
display(Markdown(filename='README.md'))

# PENDULUM
**STANDALONE TOPOLOGY**

------------------------------------------

### **Summary**
A pendulum is a weight suspended from a pivot so that it can swing freely. It consists of ywo bodies, the pendulum and the ground, attached together through a pin/revolute joint. More general information about the pendulum can be found on [wikipedia](https://en.wikipedia.org/wiki/Pendulum).

------------------------------------------

### **Topology Layout**
The mechanism consists of 1 Body + 1 Ground. Therefore, total system coordinates -including the ground- is 
$$n=n_b\times7 = 2\times7 = 14$$ 

where $n_b$ is the total number of bodies.  [^1]

The list of bodies is given below:

- Pendulum body $body$.

The system connectivity is as follows:
- Pendulum $body$ is connected to the ground by a revolute joint, resulting in constraint equations $n_{c,rev} = 5$

<br/>
<br/>

<center>

| Joint Name  | Body i         | Body j         | Joint Type | $n_c$ |
|:-----------:|:-------------- |:-------------- | ---------- | ----- |
| a           | Ground         | Pendulum       | Revolute   | 5     |

</center>

<br/>

The degrees of freedom of the system can be calculated as:
    $$n-( n_{c,rev}+n_{c,P}+n_{c,g}) = 14 - (5 + (1 \times 1) + 7) = 14 - 13 = 1$$

where the $n_{c,P}$ and $n_{c,g}$ represents the constraints due to euler-parameters normalization equations and the ground-constraints respectively.


<br/>

-------------------------------------------------------



[^1]: The tool uses [euler-parameters](https://en.wikibooks.org/wiki/Multibody_Mechanics/Euler_Parameters) -which is a 4D unit quaternion- to represents bodies orientation in space. This makes the generalized coordinates used to fully define a body in space to be **7,** instead of **6**, it also adds an algebraic equation to the constraints that ensures the unity/normalization of the body quaternion. This is an important remark as the calculations of the degrees-of-freedom depends on it.



---------------------------------------------------------------
---------------------------------------------------------------

# **IMPLEMENTATION**

The system is modeled as full multi-body system (**MBS**) using the [**uraeus.smbd**](https://https://github.com/khaledghobashy/uraeus-smbd) to model the system symbolically, then using [**uraeus.nmbd.cpp**](https://github.com/khaledghobashy/uraeus_nmbd_cpp) to perform the numerical simulation of the modeled system.

The implementation in this notebook can be broken down into main **six** steps as follows:

1. **Colab Machine Setup.** </br>
We first starts by setting up the Colab machine environment by installing the needed tools and packages.

2.  **Symbolic Model Creation.** </br>
Here we create the symbolic topology of the system as well as a symbolic configuration.

3.  **Numerical Environment Generation.** </br>
We then pass our symbolic model to code-generators to generate the code files needed for numerical simulation.

4.  **Numerical Simulation.** </br>
We then use these code files to create our simulation instances and run the numerical simulation.

5.  **Data Post-Processing.** </br>
Now, we can use the raw results' data to evaluate the required characteristics and create plots.

6.  **3D Visualization.** </br>
Finally, we use **uraeus.visenv.babylon** to visualize and animate our system in 3D.
---

*__Note__: If you want to know more about the **uraues.mbd** opensource packages for multi-body dynamics, please visit the package repository at github [here](https://github.com/khaledghobashy/uraeus_mbd).*

---
---

# 1. **Colab Machine Setup**
---

This is a simple demonesteration that walks through the installation steps of the needed software packages on a virtual linux machine provided by [Google Colaboratory](https://colab.research.google.com/). These are more or less the same steps you need to follow in order to setup the environment on your machine.

The list of the stated **uraeus.nmbd.cpp** prerequisites:
- Python 3.6+.
- [Git](https://git-scm.com/downloads), for cloning the project repository.
- [Cmake](https://cmake.org/download/), for build-systems generation. The project requires cmake 3.10 or higher.
- A modern C++ compiler supporting C++17 standards. The project is tested with the GCC 10 compiler on a Linux machine and the Microsoft C++ build tools MSVC on a Windows-10 machine.
- The [uraeus.smbd](https://github.com/khaledghobashy/uraeus-smbd) python package.

The linux machine provided by Colab already has compatible versions of **python**, **git** and **cmake**. We just need to:
1. Install a newer version of the **gcc** compiler.
2. Clone and install the [uraeus.smbd](https://github.com/khaledghobashy/uraeus-smbd) and [uraeus.nmbd.cpp](https://github.com/khaledghobashy/uraeus_nmbd_cpp) packages.

---

*__Note__: Code cells in Jupyter notebooks can accept terminal commands by including an exclamation mark (**!**) before the command. This will be used to setup the virtual machine hosting this notebook.*

---

## Installing GCC 10
The code cell below has the terminal commands needed to install **gcc** 10 and set it as the default compiler for the machine. Select the cell and press **Shift + Enter** to run the cell.

In [ ]:
!sudo apt install software-properties-common
!sudo add-apt-repository -y ppa:ubuntu-toolchain-r/test
!sudo apt install gcc-10 g++-10
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-10 90 --slave /usr/bin/g++ g++ /usr/bin/g++-10 --slave /usr/bin/gcov gcov /usr/bin/gcov-10

Checking the default version of gcc.

In [ ]:
!gcc --version

----

## Cloning uraeus packages
The code cell below has the terminal commands needed to clone the needed uraeus packages. Just run the cell.

In [ ]:
!git clone https://github.com/khaledghobashy/uraeus_nmbd_cpp.git
!git clone https://github.com/khaledghobashy/uraeus_smbd.git
!git clone https://github.com/khaledghobashy/uraeus_visenv_babylon.git

## Installing uraeus packages
The code cell below has the terminal commands needed to install the cloned packages via `pip install`. Just run the cell.

In [ ]:
!pip install -e uraeus_smbd
!pip install -e uraeus_nmbd_cpp

---

## Building the binaries of **uraeus.nmbd.cpp**
The code cell below creats a build directory and uses `cmake ..` to generate the build-system files needed to build the binaries in **Release** mode, then invokes `cmake --build .` to start the building process. Again, jsut run the cell.

In [ ]:
!cd uraeus_nmbd_cpp/uraeus/nmbd/cpp/engine/ && mkdir -p build && cd build && cmake .. -DCMAKE_BUILD_TYPE=Release && cmake --build .

---

Finally, we force the active python kernel to restart in order to have access to the intalled uraeus packages.
Running the cell below in Colab will give a warining message **"Your session crashed for an unknown reason"**, this is intentional, so do not worry.

In [ ]:
import os
os._exit(00)

---
---

</br>
</br>
</br>
</br>

---
---

# 2. **Symbolic Model Creation**
---

## **Symbolic Topology**
------------------------

In this section, we create the symbolic topology that captures the topological layout that we discussed earlier.</br>
Defining the topology is very simple. We start by importing the ```standalone_topology``` class and create a new instance that represents our symbolic model. Then we start adding the components we discussed earlier, starting by the bodies, then the joints, actuators and forces, and thats it.</br>
These components will be represented symbolically, and therefore there is no need for any numerical inputs at this step.

The system is stored in a form of a network graph that stores all the data needed for the assemblage of the system equations later. But even before the assemblage process, we can gain helpful insights about our system as well be shown.


In [ ]:
# standard library imports
import os

# getting directory of current file and specifying the directory
# where data will be saved
os.makedirs(os.path.join("model", "symenv", "data"), exist_ok=True)
#os.chdir("model")
data_dir = os.path.abspath("model/symenv/data")

In [ ]:
# uraeus imports
from uraeus.smbd.systems import standalone_topology, configuration

# ============================================================= #
#                       Symbolic Topology
# ============================================================= #

# Creating the symbolic topology as an instance of the
# standalone_topology class
project_name = 'pendulum'
sym_model = standalone_topology(project_name)

# Adding Bodies
# =============
sym_model.add_body('body')

# Adding Joints
# =============
sym_model.add_joint.revolute('a','ground','rbs_body')

### **Symbolic Characterstics**

#### Topology Graph
Visualizing the connectivity of the system as a network graph, where the nodes represent the bodies, and the edges represent the joints, forces and/or actuators between the bodies.

In [ ]:
sym_model.topology.draw_constraints_topology()

Checking the system"s number of generalized coordinates $n$ and number of constraints $n_c$.

In [ ]:
sym_model.topology.n, sym_model.topology.nc 

### **Assembling**

This is the last step of the symbolic building process, where we make the system starts the assemblage process of the governing equations, which will be used then in the code generation for the numerical simulation, and also can be used for further symbolic manipulations.</br>

*Note: The equations" notations will be discussed in the tool documentation files.*

In [ ]:
# Assembling and Saving model
sym_model.save(data_dir)
sym_model.assemble()

#### Checking the System Equations

In [ ]:
sym_model.topology.pos_equations

---------------------------------------------------------------
---------------------------------------------------------------

## **Symbolic Configuration**
---------------------------
In this step we define a symbolic configuration of our symbolic topology. As you may have noticed in the symbolic topology building step, we only cared about the **_topology_**, thats is the system bodies and their connectivity, and we did not care explicitly with how these components are configured in space.</br>

In order to create a valid numerical simulation session, we have to provide the system with its numerical configuration needed, for example, the joints" locations and orientations. The symbolic topology in its raw form will require you to manually enter all these numerical arguments, which can be cumbersome even for smaller systems. This can be checked by checking the configuration inputs of the symbolic configuration as ```sym_config.config.input_nodes```

Here we start by stating the symbolic inputs we wish to use instead of the default inputs set, and then we define the relation between these newly defined arguments and the original ones. 

**_The details of this process will be provided in the documentation._**

In [ ]:
# ============================================================= #
#                     Symbolic Configuration
# ============================================================= #

# Symbolic configuration name.
config_name = "%s_cfg"%project_name

# Symbolic configuration instance.
sym_config = configuration(config_name, sym_model)

### Configuration Inputs

In [ ]:
# Adding the desired set of UserInputs
# ====================================
sym_config.add_point.UserInput('p1')
sym_config.add_point.UserInput('p2')

sym_config.add_vector.UserInput('v')

### Configuration Releations

In [ ]:
# Defining Relations between original topology inputs
# and our desired UserInputs.
# ===================================================

# Revolute Joint (a) location and orientation
sym_config.add_relation.Equal_to('pt1_jcs_a', ('hps_p1',))
sym_config.add_relation.Equal_to('ax1_jcs_a', ('vcs_v',))

### Geometries

Here we start defining basic geometric shapes that can represents the shapes of the bodies in our system. This serves two points:
- Visualization and Animation using the provided uraeus.visenv packages.
- Evaluating the bodies inertia properties from these basic geometries instead of explicit definition.

In [ ]:
# Creating Geometries
# ===================
sym_config.add_scalar.UserInput('radius')

sym_config.add_geometry.Sphere_Geometry('body', ('hps_p2', 's_radius'))
sym_config.assign_geometry_to_body('rbs_body', 'gms_body')

### Assembling

In [ ]:
# Exporing the configuration as a JSON file
sym_config.export_JSON_file(data_dir)

---------------------------------------------------------------
---------------------------------------------------------------

# 3. **NUMERICAL ENVIRONMENT GENERATION**
This step aims to create a valid code that can be used for numerical simulation. We will use the **uraeus.nmbd.cpp** numerical environment to create a valid numerical simulation environment in C++.
Theoretically, the symbolic environment is uncoupled from the simulation environment, which opens the door to create various simulation environments that can be in any language.

## Code-Generation
Now we can use our symbolic model to generate the numerical code that can then be used for numerical simulations. For the **uraeus.nmbd.cpp** numerical environment, we import the `standalone_project` class from the `codegen` module and pass in our symbolic model.
This will generate:
- C++ source and header files representing the topology of our model.
- C++ source and header files for a `Simulation` class exposing only the needed functionalities through a minimal API, that can be used in our `main.cpp`.
- Cython wrappers for the C++ `Simulation` class so it can be accessed through python too.
- CMakeLists.txt for automated cross-platform build-systems generation using CMake, building simulation executable and python extension modules.

In [ ]:
# ============================================================= #
#                     Code Generation
# ============================================================= #

from uraeus.nmbd.cpp.codegen import standalone_project
project = standalone_project("model")
project.create_dirs()

project.write_topology_code(sym_model)

## Building Process

Generating needed source and header files from the generated cython scripts.

In [ ]:
!cd model/numenv/cpp/cython/ && cythonize simulation.pyx call_obj.pyx

### Creating main.cpp
Here we have to write our simulation routine that will be used to produce the simulation executable.
For this model, we only need to specify the name of the `json` file for the system configuration, and specify the desired time period and desired step-size.
For other models we may have to specify the user-defined functions to control motion-actuators and generic force elements defined in our model.

We use the `%%writefile <filename>` magic command to write the content of the cell to the specified file `<filename>`.

In [ ]:
%%writefile model/numenv/cpp/src/main.cpp

#include "simulation.hpp"

int main()
{
    std::cout << "Calling Solver Default Constructor\n";
    auto sim = Simulation();

    std::cout << "Calling constructFromJSON\n";
    sim.ConstructConfiguration("configuration.json");
    
    sim.Solve(10, 5e-3);

    sim.SaveResults("", "free_dynamics");

    return 0;
};


### Building Simulation Binaries
The code cell below invokes `cmake ..` to generate the build-system files needed to build the binaries in **Release** mode, then invokes `cmake --build .` to start the building process.

This will produce a `model` executable and a `simulation.so` python extension module that can be imported as a normal python module. These will be placed in the `/bin` directory

In [ ]:
!cd model/numenv/cpp/build && cmake .. -DCMAKE_BUILD_TYPE=Release && cmake --build .

---
---

# 4. **NUMERICAL SIMULATION**
Now we can advance to the **nuemrical simulation** step. In order to perform our desired simulation, we should first define our model numerical configuration data. These data represents how our model is placed in 3D space, where the bodies are located and how they are oriented as well as the other physical components existing in the system.

At the **Symbolic Configuration** step we defined some **user inputs** and some **relations** that states how our system is configured symbolically. These data is stored in a `.json` file at the `/symenv/data/` directory. All we have to do is to make a copy of this file and fill in our numerical configuration data for our defined `user_inputs`.

This file should be passed to our executable to construct the numerical topology correctly.

In order to provide the numerical configuration data, we have to write a `configuration.json` file and save it at `numenv/cpp/bin/`. Using the `%pycat <filename>` magic command in Colab will open a side tab containing the file content, we can copy the content and past it in anotehr code cell and apply our modifications, then we use the `%%writefile <filename>` magic command to creat the required `configuration.json` file.

Another -more elegant- option is to make use of python's `json` module to load, edit and save `.json` files. The `json` module loads json formatted text as python dictionaries, where we can access and modify our data easily. Then we can use the `json` module again to `dump` our modified data dictionary as json formatted text again and save it to a new file.

In [ ]:
%pycat model/symenv/data/pendulum_cfg.json

Loading the configuration data using `json` module, and getting access to the `user_inputs` section.

In [ ]:
import json

with open("model/symenv/data/pendulum_cfg.json", "r") as f:
    json_text = f.read()

data = json.loads(json_text)
user_inputs = data["user_inputs"]

Filling in our numerical configuration data

In [ ]:

# Mechanism Points
# ================
user_inputs["hps_p1"]["args"] = [ 0,   0, 0]
user_inputs["hps_p2"]["args"] = [ 0, 200, 0]


# Guiding Global Axes
# ===================
user_inputs["vcs_v"]["args"] = [1, 0, 0]

# Scalar Variables
# ================
user_inputs["s_radius"] = 20


Saving our numerical configuration as `configuration.json` at the required location `numenv/cpp/bin/`.

In [ ]:
modified_text = json.dumps(data, indent=4)

with open("model/numenv/cpp/bin/configuration.json", "w") as output:
    output.write(modified_text)

### Run the Simulation Executable

In [ ]:
!cd model/numenv/cpp/bin/ && ./model

### Test the generated python extension module

In [ ]:
%%writefile model/numenv/cpp/bin/main.py

import numpy as np
from simulation import PySimulation

# creating a model instance of the wrapped simulation class
model = PySimulation()

# constructing the numerical configuration from the modified
# json file.
model.construct_configuration("configuration.json")

# setting the simulation duration and time step-size
model.solve(10, 5*1e-3)

# saving the simulation results
model.save_results("", "py_free_dynamics")

### Run main.py python script

In [ ]:
!cd model/numenv/cpp/bin/ && python main.py

---
---

# 5. **DATA POST-PROCESSING**
We can load and plot the simulation results easily too. The `SaveResults` method exports the position, velocities and accelerations of the system generalized coordinates as a `name_pos.csv`, `name_vel.csv`, and `name_acc.csv` respectively. These files can be loaded into a `pandas.DataFrame` object easily where we can do what we wish with the data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pos_data = pd.read_csv("model/numenv/cpp/bin/py_free_dynamics_pos.csv", index_col=0)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(pos_data['time'], pos_data['rbs_body.z'], label='rbs_body.z')
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(pos_data['time'], pos_data['rbs_body.y'], label='rbs_body.y')
plt.grid()
plt.legend()
plt.show()

# 6. **3D Visualization**
An extra step we can do here is to visualize and animate our system in 3D. This can be done using the **uraeus.visenv.babylon** visualization environment, which is a JavaScript-based WebGL visualization environment for visualizing multi-body models created using uraeus.

Making use of the fact that Colab is fully operational virtual linux machine, we can start a python server in the background running the visualization engine, and make use of the jupyter notebook capabilities to embed an iframe inside cell's output.

Just run the cell below, and it should render the visualization environment.

*__Note__: In Colab, you can right-click on a given cell and select **view output fullscreen**, this will open the visualization environment in a fullscreen mode.*

*__Note__: You have to download the `configuration.json` and the `_pos.csv` from the `numenv/cpp/bin/` directory to your machine, as the **uraeus.visenv.babylon** will prompt a window asking for these files on your physical machine when you try to load a model and/or an animation file.*

The method below assumes a Colab session. This will not work on your local machine.

In [ ]:
import subprocess
process = subprocess.Popen("cd uraeus_visenv_babylon && python -m http.server 8000",
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

In [ ]:
from google.colab.output import eval_js
print("Click on the link below to open the 'visualization window' in a new tab.")
print(eval_js("google.colab.kernel.proxyPort(8000)"))

Another way to view the 'visualization window' is to embed it as an `iframe` inside a cell output as below.

In [ ]:
from IPython.display import IFrame
IFrame(eval_js("google.colab.kernel.proxyPort(8000)"), 1200, 500)